In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os
import pandas as pd
#Functional coding
import functools
from functools import partial
from tensorflow.python.ops import array_ops 

In [2]:
#Given local path, find full path
def PathFinder(path):
    #python 2
    #script_dir = os.path.dirname('__file__')
    #fullpath = os.path.join(script_dir,path)
    #python 3
    fullpath=os.path.abspath(path)
    print(fullpath)
    return fullpath

#Read Data for Deep Learning
def ReadData(path):
    fullpath=PathFinder(path)
    return pd.read_csv(fullpath, sep=',', header=0)

In [3]:
default_settings = {
    "max_cap"          : [100,100,100,100,100,100,100,100,200,200,200,200,200,200,400,400,400,500,500,1000],
    "high_bound"       : [80,80,80,80,80,80,80,80,180,180,180,180,180,180,380,380,380,480,480,980],
    "low_bound"        : [20,20,20,20,20,20,20,20,30,30,30,30,30,30,40,40,40,60,60,100],
    "rain"             : [5,5,5,5,5,5,5,5,10,10,10,10,10,10,20,20,20,30,30,40],
    "downstream"       : [[1,9],[2,9],[3,10],[4,10],[5,11],[6,11],[7,12],[8,12],[9,15],[10,15],\
                          [11,16],[12,16],[13,17],[14,17],[15,18],[16,19],[17,19],[18,20],[19,20]],
    "downtosea"        : [20],
    "biggestmaxcap"    : 1000,
    "reservoirs"    : [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
    "init_state"       : [75,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50]
   }


In [4]:
class RESERVOIR(object):
    def __init__(self, 
                 batch_size,
                 default_settings):
        self.batch_size = batch_size
        self.reservoirs = default_settings['reservoirs']
        self.reservoir_num = len(default_settings['reservoirs'])
        self.biggestmaxcap = tf.constant(default_settings["biggestmaxcap"],dtype=tf.float32)
        self.zero = tf.constant(0,shape=[self.batch_size,self.reservoir_num],dtype=tf.float32)
        self._high_bounds(default_settings["high_bound"])
        self._low_bounds(default_settings["low_bound"])
        self._rains(default_settings["rain"])
        self._max_cap(default_settings["max_cap"])
        self._downstream(default_settings["downstream"])
        self._downtosea(default_settings["downtosea"])
        
    def _max_cap(self, max_cap_list):
        self.max_cap = tf.constant(max_cap_list,dtype=tf.float32)
    
    def _high_bounds(self, high_bound_list):
        self.high_bound = tf.constant(high_bound_list,dtype=tf.float32)
            
    def _low_bounds(self, low_bound_list):
        self.low_bound = tf.constant(low_bound_list,dtype=tf.float32)
            
    def _rains(self, rain_list):
        self.rain = tf.constant(rain_list,dtype=tf.float32)
        
    def _downstream(self, downstream):
        np_downstream = np.zeros((self.reservoir_num,self.reservoir_num))
        for i in downstream:
            m = self.reservoirs.index(i[0])
            n = self.reservoirs.index(i[1])
            np_downstream[m,n] = 1
        self.downstream = tf.constant(np_downstream,dtype=tf.float32)
        
    def _downtosea(self, downtosea):
        np_downtosea = np.zeros((self.reservoir_num,))
        for i in downtosea:
            m = self.reservoirs.index(i)
            np_downtosea[m] = 1
        self.downtosea =  tf.constant(np_downtosea,dtype=tf.float32)
            
    def MAXCAP(self):
        return self.max_cap
    
    def HIGH_BOUND(self):
        return self.high_bound
    
    def LOW_BOUND(self):
        return self.low_bound
    
    def RAIN(self):
        return self.rain
    
    def DOWNSTREAM(self):
        return self.downstream
    
    def DOWNTOSEA(self):
        return self.downtosea
        
    def BIGGESTMAXCAP(self):
        return self.biggestmaxcap
    
    def Transition(self, states, actions):
        previous_state = states
        vaporated = 0.5*tf.sin(previous_state/self.BIGGESTMAXCAP())*previous_state
        upstreamflow = tf.transpose(tf.matmul(tf.transpose(self.DOWNSTREAM()),tf.transpose(actions)))
        new_state = previous_state + self.RAIN()-vaporated-actions+ upstreamflow                        
        return new_state
    
    #Reward for Reservoir is computed on 'Next State'
    def Reward(self, states):
        new_rewards = tf.select(tf.logical_and(tf.greater_equal(states,self.LOW_BOUND()),tf.less_equal(states,self.HIGH_BOUND())),\
                                 self.zero,\
                                tf.select(tf.less(states,self.LOW_BOUND()),\
                                          -5*(self.LOW_BOUND()-states),\
                                         -100*(states-self.HIGH_BOUND()))\
                               )
        new_rewards+=tf.abs(((self.HIGH_BOUND()+self.LOW_BOUND())/2.0)-states)*(-0.1)
        return tf.reduce_sum(new_rewards,1,keep_dims=True)       

In [5]:
class RESERVOIRCell(tf.nn.rnn_cell.RNNCell):

    def __init__(self, batch_size,default_settings):
        self._num_state_units = len(default_settings["reservoirs"])
        self._num_reward_units = self._num_state_units +1
        self.reservoir = RESERVOIR(batch_size,default_settings)

    @property
    def state_size(self):
        return self._num_state_units

    @property
    def output_size(self):
        return self._num_reward_units

    def __call__(self, inputs, state, scope=None):
        next_state =  self.reservoir.Transition(state, inputs)
        reward = self.reservoir.Reward(next_state)   
        return tf.concat(1,[reward,next_state]), next_state

In [6]:
class ActionOptimizer(object):
    def __init__(self,
                a, # Actions
                num_step, # Number of RNN step, this is a fixed step RNN sequence, 12 for navigation
                num_act, # Number of actions
                batch_size, #Batch Size
                loss,
                learning_rate=0.1): 
        self.action = a
        print(self.action)
        self.batch_size = batch_size
        self.num_step = num_step
        self.learning_rate = learning_rate
        self._p_create_rnn_graph()
        if loss == "Qloss":
            self._p_Q_loss()
        else:
            self._p_create_loss()
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())
    
    def _p_create_rnn_graph(self):
        cell = RESERVOIRCell(self.batch_size, default_settings)
        initial_state = cell.zero_state(self.batch_size, dtype=tf.float32)+tf.constant([default_settings["init_state"]],dtype=tf.float32)
        print('action batch size:{0}'.format(array_ops.shape(self.action)[0]))
        print('Initial_state shape:{0}'.format(initial_state))
        rnn_outputs, state = tf.nn.dynamic_rnn(cell, self.action, dtype=tf.float32,initial_state=initial_state)
        #need output intermediate states as well
        concated = tf.concat(0,rnn_outputs)
        print('concated shape:{0}'.format(concated.get_shape()))
        something_unpacked =  tf.unpack(concated, axis=2)
        self.outputs = tf.reshape(something_unpacked[0],[-1,self.num_step,1])
        print(' self.outputs:{0}'.format(self.outputs.get_shape()))
        self.intern_states = tf.pack([something_unpacked[i+1] for i in range(len(default_settings["reservoirs"]))], axis=2)
        self.last_state = state
        self.pred = tf.reduce_sum(self.outputs,1)
        self.average_pred = tf.reduce_mean(self.pred)
        print("self.pred:{0}".format(self.pred))
            
    def _p_create_loss(self):
        print("MSE-loss")
        objective = tf.reduce_mean(tf.square(self.pred)) 
        self.loss = objective
        print(self.loss.get_shape())
        #self.loss = -objective
        self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss, var_list=[a])
        
    def _p_Q_loss(self):
        print("Q-loss")
        objective = tf.constant(0.0, shape=[self.batch_size, 1])
        for i in range(self.num_step):
            Rt = self.outputs[:,i]
            SumRj=tf.constant(0.0, shape=[self.batch_size, 1])
            #SumRk=tf.constant(0.0, shape=[self.batch_size, 1])
            if i<(self.num_step-1):
                j = i+1
                SumRj = tf.reduce_sum(self.outputs[:,j:],1)
            #if i<(self.num_step-1):
                #k= i+1
                #SumRk = tf.reduce_sum(self.outputs[:,k:],1)
            objective+=(Rt*SumRj+tf.square(Rt))*(self.num_step-i)/np.square(self.num_step)
            #objective+=(Rt*SumRj+tf.square(Rt))/(self.num_step-i)
        self.loss = tf.reduce_mean(objective)
        self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss, var_list=[a])
        
    def Optimize(self,epoch=100):       
#         Time_Target_List = [15,30]
#         Target = Time_Target_List[0]
#         counter = 0
#         new_loss = self.sess.run([self.average_pred])
#         print('Loss in epoch {0}: {1}'.format("Initial", new_loss)) 
#         print('Compile to backend complete!') 
#         start = time.time()
#         while True:
#             training = self.sess.run([self.optimizer])
#             action_upperbound=self.sess.run(self.intern_states)
#             self.sess.run(tf.assign(self.action, tf.clip_by_value(self.action, 0, action_upperbound)))
#             end = time.time()
#             if end-start>=Target:
#                 print('Time: {0}'.format(Target))
#                 pred_list = self.sess.run(self.pred)
#                 pred_list=np.sort(pred_list.flatten())[::-1]
#                 pred_list=pred_list[:5]
#                 pred_mean = np.mean(pred_list)
#                 pred_std = np.std(pred_list)
#                 print('Best Cost: {0}'.format(pred_list[0]))
#                 print('MEAN: {0}, STD:{1}'.format(pred_mean,pred_std))
#                 counter = counter+1
#                 if counter == len(Time_Target_List):
#                     print("Done!")
#                     break
#                 else:
#                     Target = Time_Target_List[counter]
        new_loss = self.sess.run([self.loss])
        print('Loss in epoch {0}: {1}'.format("Initial", new_loss)) 
        start_time = time.time()
        for epoch in range(epoch):
            training = self.sess.run([self.optimizer])
            action_upperbound=self.sess.run(self.intern_states)
            self.sess.run(tf.assign(self.action, tf.clip_by_value(self.action, 0, action_upperbound)))
            if True:
                new_loss = self.sess.run([self.loss])
                print('Loss in epoch {0}: {1}'.format(epoch, new_loss))  
        minimum_costs_id=self.sess.run(tf.argmax(self.pred,0))
        print(minimum_costs_id)
        best_action = np.round(self.sess.run(self.action)[minimum_costs_id[0]],4)
        print('Optimal Action Squence:{0}'.format(best_action))
        print('Best Cost: {0}'.format(self.sess.run(self.pred)[minimum_costs_id[0]]))
        pred_list = self.sess.run(self.pred)
        pred_list=np.sort(pred_list.flatten())[::-1]
        pred_list=pred_list[:5]
        pred_mean = np.mean(pred_list)
        pred_std = np.std(pred_list)
        print('Best Cost: {0}'.format(pred_list[0]))
        print('Sorted Costs:{0}'.format(pred_list))
        print('MEAN: {0}, STD:{1}'.format(pred_mean,pred_std))
        print('The last state:{0}'.format(self.sess.run(self.last_state)[minimum_costs_id[0]]))
        print('Rewards each time step:{0}'.format(self.sess.run(self.outputs)[minimum_costs_id[0]]))
        print('Intermediate states:{0}'.format(self.sess.run(self.intern_states)[minimum_costs_id[0]]))

In [7]:
sess = tf.InteractiveSession()
initial_a = tf.truncated_normal(shape=[100,120,20],mean=0.0, stddev=0.5).eval() 
a = tf.Variable(initial_a,name="action")
rnn_inst = ActionOptimizer(a, 120,20,100,"MSE")  

Tensor("action/read:0", shape=(100, 120, 20), dtype=float32)
action batch size:Tensor("strided_slice:0", shape=(), dtype=int32)
Initial_state shape:Tensor("add:0", shape=(100, 20), dtype=float32)
concated shape:(100, 120, 21)
 self.outputs:(100, 120, 1)
self.pred:Tensor("Sum:0", shape=(100, 1), dtype=float32)
MSE-loss
()


In [8]:
rnn_inst.Optimize(4000)

Loss in epoch Initial: [2.6633186e+12]
Loss in epoch 0: [1.673244e+12]
Loss in epoch 1: [1.2236447e+12]
Loss in epoch 2: [9.1653204e+11]
Loss in epoch 3: [6.925043e+11]
Loss in epoch 4: [5.2393463e+11]
Loss in epoch 5: [3.9518958e+11]
Loss in epoch 6: [2.962781e+11]
Loss in epoch 7: [2.2025557e+11]
Loss in epoch 8: [1.6206329e+11]
Loss in epoch 9: [1.1786252e+11]
Loss in epoch 10: [8.4628177e+10]
Loss in epoch 11: [5.9922919e+10]
Loss in epoch 12: [4.1833447e+10]
Loss in epoch 13: [2.8822602e+10]
Loss in epoch 14: [1.9662713e+10]
Loss in epoch 15: [1.3355613e+10]
Loss in epoch 16: [9.1083428e+09]
Loss in epoch 17: [6.291967e+09]
Loss in epoch 18: [4.4300186e+09]
Loss in epoch 19: [3.190956e+09]
Loss in epoch 20: [2.3549404e+09]
Loss in epoch 21: [1.7804614e+09]
Loss in epoch 22: [1.3770004e+09]
Loss in epoch 23: [1.087855e+09]
Loss in epoch 24: [8.7534003e+08]
Loss in epoch 25: [7.1697286e+08]
Loss in epoch 26: [5.9642534e+08]
Loss in epoch 27: [5.0338858e+08]
Loss in epoch 28: [4.3049